In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
# import SVD
# import accuracy

import warnings; warnings.simplefilter('ignore')

In [2]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [3]:
import json
import pandas as pd
import numpy as np

import requests

from tqdm import tqdm_notebook
from collections import Counter

# 청소년 - 책 소개 기반 추천

- 데이터불러오기

In [83]:
# excel 불러오기
teenager_book_revised=pd.read_excel('./data/final/content_re/teenager_book_revised.xlsx')

In [19]:
teenager_book_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                978 non-null    int64  
 1   ranking           978 non-null    int64  
 2   bookname          978 non-null    object 
 3   authors           978 non-null    object 
 4   publisher         978 non-null    object 
 5   publication_year  913 non-null    float64
 6   isbn13            978 non-null    int64  
 7   addition_symbol   916 non-null    float64
 8   vol               181 non-null    float64
 9   class_no          916 non-null    float64
 10  loan_count        978 non-null    int64  
 11  bookImageURL      978 non-null    object 
 12  keyword           540 non-null    object 
 13  describe          978 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 107.1+ KB


In [84]:
# 제목 양쪽 공백제거
for i in range(len(teenager_book_revised)):
    teenager_book_revised['bookname'][i]=str(teenager_book_revised['bookname'][i]).strip()

In [86]:
# 책소개가 없는 데이터지우기
teenager_book_revised=teenager_book_revised[pd.notnull(teenager_book_revised['describe'])]

In [85]:
# 책 제목 중복 제거 
teenager_book_revised.drop_duplicates('bookname',keep='first',inplace=True)

In [87]:
# 인덱스 정리
teenager_book_revised.reset_index(drop=True,inplace=True)

In [17]:
# 형용사 + 명사 기반

# book_intro=[]
# for i in range(len(teenager_book_revised)):
#     sentences_tag = []
#     lines=teenager_book_revised['describe'][i].replace('\n',' ').split('.')
#     twitter = Twitter()
#     for sentence in lines:
#         morph = twitter.pos(sentence)
#         sentences_tag.append(morph)
#     noun_adj_list = []
#     for sentence1 in sentences_tag:
#         for word, tag in sentence1:
#             if tag in ['Noun','Adjective']:
#                 noun_adj_list.append(word)
#     book_intro.append(noun_adj_list)

In [88]:
teenager_book_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   no                874 non-null    int64  
 1   ranking           874 non-null    int64  
 2   bookname          874 non-null    object 
 3   authors           874 non-null    object 
 4   publisher         874 non-null    object 
 5   publication_year  858 non-null    float64
 6   isbn13            874 non-null    int64  
 7   addition_symbol   813 non-null    float64
 8   vol               99 non-null     float64
 9   class_no          813 non-null    float64
 10  loan_count        874 non-null    int64  
 11  bookImageURL      874 non-null    object 
 12  keyword           534 non-null    object 
 13  describe          874 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 95.7+ KB


In [105]:
# 데이터 저장 
teenager_book_revised.to_csv('./data/final/content_re/teenager_final.csv',index=False,encoding='utf-8-sig')

# TF-IDF

In [90]:
# 블용어 txt파일 리스트 가져오기
sw=pd.read_csv('./data/final/한국어불용어100.txt', sep = "\t",  engine='python', encoding = "utf-8",header=None)

In [91]:
stop_words=list(sw[0].dropna())

In [92]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(teenager_book_revised['describe'])

In [93]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [94]:
titles = teenager_book_revised['bookname']
indices = pd.Series(teenager_book_revised.index, index=teenager_book_revised['bookname'])

In [95]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [96]:
# ex1
get_recommendations('나미야 잡화점의 기적 :히가시노 게이고 장편소설').head(10)

286      녹나무의 파수꾼 =The camphorwood custodian
384                              범인 없는 살인의 밤
361                                  교통경찰의 밤
179                        악의 :히가시노 게이고 장편소설
270             운다고 달라지는 일은 아무것도 없겠지만 :박준 산문
21                   라플라스의 마녀 :히가시노 게이고 장편소설
741    내가 원하는 것을 나도 모를 때 :잃어버린 나를 찾는 인생의 문장들
37                                간호사라서 다행이야
181                                11문자 살인사건
137          방구석 미술관 :가볍고 편하게 시작하는 유쾌한 교양 미술
Name: bookname, dtype: object

In [98]:
# ex2
get_recommendations('지적 대화를 위한 넓고 얕은 지식 0 :지혜를 찾아 138억 년을 달리는 시간 여행서').head(10)

433               시민의 교양 :지금, 여기, 보통 사람들을 위한 현실 인문학
186                              지적 대화를 위한 넓고 얕은 지식
490                        열한 계단 :나를 흔들어 키운 불편한 지식들
472    우리 몸이 세계라면 :분투하고 경합하며 전복되는 우리 몸을 둘러싼 지식의 사회사
451                 (1일 1페이지, 세상에서 가장 짧은) 교양 수업 365
772                            (재밌어서 밤새 읽는) 천문학 이야기
530                               혼자가 혼자에게 :이병률 산문집
864                                관계의 물리학 :림태주 에세이
741           내가 원하는 것을 나도 모를 때 :잃어버린 나를 찾는 인생의 문장들
39                    설민석의 조선왕조실록 :대한민국이 선택한 역사 이야기
Name: bookname, dtype: object

In [100]:
# ex3
get_recommendations('곰돌이 푸, 서두르지 않아도 괜찮아 :곰돌이 푸 원작').head(10)

169            곰돌이 푸, 행복한 일은 매일 있어 :아직 행복을 기다리는 우리에게
131                             12가지 인생의 법칙 :혼돈의 해독제
159                          꽃을 보듯 너를 본다 :나태주 인터넷 시집
437                              딸에게 주는 레시피 :공지영 에세이
238    하고 싶은 대로 살아도 괜찮아 :늘 남에게 애쓰기만 하느라 나를 잃어버린 당신에게
209                                      모리와 함께한 화요일
801                                오래된 미래:라다크로부터 배우다
178                       책을 지키려는 고양이 :나쓰카와 소스케 장편소설
103                                 물 만난 물고기 :이찬혁 소설
847               수학의 쓸모 :불확실한 미래에서 보통 사람들도 답을 얻는 방법
Name: bookname, dtype: object

In [102]:
get_recommendations('파피용 :베르나르 베르베르 장편소설').head(10)

132                                 개미 :베르나르 베르베르 장편소설
355                                                 나무
276                               제3인류 :베르나르 베르베르 장편소설
838                                              데스 큐어
179                                  악의 :히가시노 게이고 장편소설
76                                  죽음 :베르나르 베르베르 장편소설
452    지적 대화를 위한 넓고 얕은 지식 0 :지혜를 찾아 138억 년을 달리는 시간 여행서
481                                 기억 :베르나르 베르베르 장편소설
257                                  원미동 사람들 :양귀자 연작소설
187                                 목소리를 드릴게요 :정세랑 소설집
Name: bookname, dtype: object

In [62]:
get_recommendations('조선왕조실톡 2 - 조선 패밀리의 활극').head(5)

104    강성태 66일 공부법 - 어떤 시험도 단박에 성적을 올리는 고효율 공부 습관
333      공부 마스터 플랜 :서울대 100인이 매일 실천한 60가지 체크 리스트 
394                             영어 1등급은 이렇게 공부한다 
445           압축 공부 - 공부의 방향을 잡아 속도를 올려주는 1등급의 법칙
330                 가짜 1등 배동구 - 박철범의 국내 최초 공부법 소설
Name: bookname, dtype: object

In [103]:
get_recommendations('내게 무해한 사람 :최은영 소설').head(10)

31                 쇼코의 미소 :최은영 소설
24                 바깥은 여름 :김애란 소설
373              현남 오빠에게 :페미니즘 소설
222               오직 두 사람 :김영하 소설
33     우리가 빛의 속도로 갈 수 없다면 :김초엽 소설
509              마르타의 일 :박서련 장편소설
232                    숨 :테드 창 소설
64             대도시의 사랑법 :박상영 연작소설
225             다이어트 학교 :김혜정 장편소설
111      그대 눈동자에 건배 :히가시노 게이고 소설집
Name: bookname, dtype: object